#Import

In [24]:
#!pip install tensorflow
import pandas as pd

In [25]:
from transformers import pipeline

pipe = pipeline("fill-mask", model="distilbert/distilbert-base-multilingual-cased") # pipeline

intentions = pd.read_csv("/content/test_intent_dataset.csv") # intents

Device set to use cpu


#Model

In [37]:
display(intentions)

,text,intent
0,Hello,greeting
1,Hi there!,greeting
2,Good morning,greeting
3,Hey,greeting
4,Howdy!,greeting
...,...,...
67,I want to know about your products,product_info
68,Give me details about your packages,product_info
69,Do you have any discounts?,product_info
70,What are your latest offers?,product_info


In [49]:
from transformers import AutoTokenizer

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

tokens = tokenizer(
    intentions["text"].tolist(),
    padding=True,
    truncation=True,
    max_length=64,
    return_tensors="tf"
)

features = { name: tensor for name, tensor in tokens.items() }

In [50]:
from sklearn.preprocessing import LabelEncoder

# Convert intention labels into numerical form
label_encoder = LabelEncoder()
intentions['encoded_labels'] = label_encoder.fit_transform(intentions['intent'])

labels = intentions["encoded_labels"].values

In [51]:
# Token /encoded label (text/intent) dataset for training
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
dataset = dataset.shuffle(1000).batch(8)

In [53]:
# Model training
from transformers import TFDistilBertForSequenceClassification
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert/distilbert-base-multilingual-cased", num_labels=len(intentions['encoded_labels'].unique()))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

# Train the model
model.fit(dataset, epochs=3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
9/9 [==============================] - 51s 3s/step - loss: 2.4293 - accuracy: 0.1389
Epoch 2/3
9/9 [==============================] - 28s 3s/step - loss: 2.2073 - accuracy: 0.4167
Epoch 3/3
9/9 [==============================] - 28s 3s/step - loss: 1.9443 - accuracy: 0.6389
